# Direct and inverse geometry of 2d robots

This notebook the main concept of kinematic tree, direct geometry and inverse geometry, but without the kinematic tree of Pinocchio. We only use the basic geometries of the Gepetto Viewer for displaying the simple robot that is used in this tutorial.

In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need NumPy, SciPy, and Gepetto Viewer for vizualizing the robot.
Scipy is a collection of scientific tools for Python. It contains, in particular, a set of optimizers that we are going to use for solving the inverse-geometry problem. If not done yet, install scipy with `sudo apt-get install python3-scipy`

In [2]:
import time
import numpy as np
from scipy.optimize import fmin_bfgs,fmin_slsqp
import gviewserver

## Displaying objects

Let's first learn how to open a 3D viewer, in which we will build our simulator. First start gepetto-gui. Best is to run it directly from the shell by typing gepetto-gui. A new window with the Gepetto logo should open. Objects can be now created from the python commands.

The following GView object is a client of the Gepetto Viewer server, i.e. it will be use to pass display command to the viewer. The first commands are to create objects, 

In [3]:
gv = gviewserver.GepettoViewerServer()

gv.addSphere('world/ball', .1, [1, 0, 0, 1])  # radius, color=[r,g,b,1]
gv.addCapsule('world/capsule', .05,.75, [1, 1, 1, 1])  # radius, length, color = [r,g,b,a]
gv.addBox('world/box', .2,.05,.5,  [.5, .5, 1, 1]);  # depth(x), length(y), height(z), color

Execute the above python commands once, you get a "True" output. Execute it a second time, you get a False: that's just telling you that the object world/box already exists and Gepetto viewer cannot create it again. If you want to erase your world and all your objects, just run:


In [4]:
gv.deleteNode('world', True)  # name, all=True

After that, the link toward the viewer server "gv" is broken. You now have to run again the gviewerserver.GepettoViewerServer() command to create the world again.


In [5]:
gv = gviewserver.GepettoViewerServer()

Placing objects can be done using the applyConfiguration command, and specifying the placement as a 3D translation and quaternion rotation. Don't forget to refresh your window after placing your objects.

In [6]:
gv.applyConfiguration('world/box', [.1, .1, .1, 1, 0, 0, 0])  # x, y, z, quaternion
gv.refresh()

In a first time, we will work in 2D. Here is a shortcut to place an object from x,y,theta 2d placement.

In [7]:
def placement(x, y, theta): 
    return [y, 0, x, 0, np.sin(theta / 2), 0, np.cos(theta / 2)]

An example of a shorter positioning of a 2D object using this shortcut is:

In [8]:
gv.applyConfiguration('world/capsule', placement(0.1, 0.2, np.pi / 4))
gv.refresh()

## Creating a 2d robot
This robot will have 2 joints, named shoulder and elbow, with link of length 1 to connect them. First let's create the 5 geometry objects.

In [9]:
gv.addSphere ('world/joint1', .1, [1 ,0 ,0,1])
gv.addSphere ('world/joint2', .1, [1 ,0 ,0,1])
gv.addSphere ('world/joint3', .1, [1 ,0 ,0,1])
gv.addCapsule('world/arm1', .05, .75, [1 ,1 ,1,1])
gv.addCapsule('world/arm2', .05, .75, [1 ,1 ,1,1]);

Given a configuration vector q of dimension 2, compute the position of the centers of each object, and display correctly the robot.

In [10]:
q = np.matrix(np.random.rand(2) * 6 - 3).T

In [12]:
# %load -r 23-35 tp1/configuration_reduced.py
def display(q):
    '''Display the robot in Gepetto Viewer. '''
    assert (q.shape == (2, 1))
    c0 = np.cos(q[0, 0])
    s0 = np.sin(q[0, 0])
    c1 = np.cos(q[0, 0] + q[1, 0])
    s1 = np.sin(q[0, 0] + q[1, 0])
    gv.applyConfiguration('world/joint1', placement(0, 0, 0))
    gv.applyConfiguration('world/arm1', placement(c0 / 2, s0 / 2, q[0, 0]))
    gv.applyConfiguration('world/joint2', placement(c0, s0, q[0, 0]))
    gv.applyConfiguration('world/arm2', placement(c0 + c1 / 2, s0 + s1 / 2, q[0, 0] + q[1, 0]))
    gv.applyConfiguration('world/joint3', placement(c0 + c1, s0 + s1, q[0, 0] + q[1, 0]))
    gv.refresh()

In [14]:
display(q) # Display the robot in Gepetto Viewer

The end effector is already computed in the previous method. Let's build a dedicated method to return the same value.

In [16]:
# %load -r 37-44 tp1/configuration_reduced.py
def endeffector(q):
    '''Return the 2D position of the end effector of the robot at configuration q. '''
    assert (q.shape == (2, 1))
    c0 = np.cos(q[0, 0])
    s0 = np.sin(q[0, 0])
    c1 = np.cos(q[0, 0] + q[1, 0])
    s1 = np.sin(q[0, 0] + q[1, 0])
    return np.matrix([c0 + c1, s0 + s1]).T

In [17]:
endeffector(q)

matrix([[-1.94985342],
        [-0.4398189 ]])

## Optimize the configuration 

Optimization will be done with the BFGS solver of scipy, which simply takes an intial guess and a cost function. Here the cost will be the squared distance to a given target.

In [20]:
# %load -r 46-51 tp1/configuration_reduced.py
target = np.matrix([.5, .5]).T

def cost(q):
    q = np.matrix(q).T
    eff = endeffector(q)
    return np.linalg.norm(eff - target)**2

In SciPy, BFGS also accepts a callback function, that we will use to display in the viewer the current value of the decision variable.

In [27]:
# %load -r 53-56 tp1/configuration_reduced.py
def callback(q):
    q = np.matrix(q).T
    display(q)
    time.sleep(.5)

And that is it, let's call BFGS.

In [28]:
# %load -r 59-60 tp1/configuration_reduced.py
x0 = np.array([0.0, 0.0])
xopt_bfgs = fmin_bfgs(cost, x0, callback=callback)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 9
         Function evaluations: 60
         Gradient evaluations: 15


## What configuration to optimize?
It seems logical to optimize over the angles $q_1,q_2$. However, other representations of the configuration are possible. Consider for example the explicit representation, where the placement of each body 1,2,3 is stored. For each body, we get $x,y,\theta$, so 9 parameters in total. In addition, each body position is constrained with respect to the placement of the previous body, with 6 constraints in total. 

What are the pros and cons? The effector position is now a trivial function of the representation, hence the cost function is very simple. The trade-off is that we have to explicitly satisfy the constraints. 

Let's start by defining the configuration.

In [29]:
x1, y1, th1, x2, y2, th2, x3, y3, th3 = x0 = np.zeros(9)

The cost function is now just a sparse difference on x3,y3:

In [32]:
# %load -r 34-37 tp1/configuration_extended.py
def endeffector_9(ps):
    assert (ps.shape == (9, ))
    x1, y1, t1, x2, y2, t2, x3, y3, t3 = ps
    return np.matrix([x3, y3]).T

In [35]:
# %load -r 41-43 tp1/configuration_extended.py
def cost_9(ps):
    eff = endeffector_9(ps)
    return np.linalg.norm(eff - target)**2

The constraint function should return a vector, each coefficient corresponding to one of the 6 constraints:

In [37]:
# %load -r 45-55 tp1/configuration_extended.py
def constraint_9(ps):
    assert (ps.shape == (9, ))
    x1, y1, t1, x2, y2, t2, x3, y3, t3 = ps
    res = np.zeros(6)
    res[0] = x1 - 0
    res[1] = y1 - 0
    res[2] = x1 + np.cos(t1) - x2
    res[3] = y1 + np.sin(t1) - y2
    res[4] = x2 + np.cos(t2) - x3
    res[5] = y2 + np.sin(t2) - y3
    return res

For example, the configuration with the 9-vector set to 0 is not satisfying the constraints.

In [38]:
print(cost_9(x0), constraint_9(x0))

0.5000000000000001 [0. 0. 1. 0. 1. 0.]


We can similarly redefined the display function and the callback

In [40]:
# %load -r 23-32 tp1/configuration_extended.py
def display_9(ps):
    '''Display the robot in Gepetto Viewer. '''
    assert (ps.shape == (9, ))
    x1, y1, t1, x2, y2, t2, x3, y3, t3 = ps
    gv.applyConfiguration('world/joint1', placement(x1, y1, t1))
    gv.applyConfiguration('world/arm1', placement(x1 + np.cos(t1) / 2, x1 + np.sin(t1) / 2, t1))
    gv.applyConfiguration('world/joint2', placement(x2, y2, t2))
    gv.applyConfiguration('world/arm2', placement(x2 + np.cos(t2) / 2, y2 + np.sin(t2) / 2, t2))
    gv.applyConfiguration('world/joint3', placement(x3, y3, t3))
    gv.refresh()

In [43]:
# %load -r 60-62 tp1/configuration_extended.py
def callback_9(ps):
    display_9(ps)
    time.sleep(.5)

### Solve with a penalty cost
The BFGS solver defined above cannot be used directly to optimize over equality constraints. A dirty trick is to add the constraint as a penalty, i.e. a high-weigth term in the cost function: $penalty(x) = cost(x) + 100*||constraint(x)||^2$ . Here, we are in a good case where the optimum corresponds to the 0 of both the constraint and the cost. The penalty with any weight would lead to the optimum and perfect constraint satisfaction. Yet the solver suffers to reach the optimum, because of the way we have described the constraint.

In [45]:
# %load -r 57-58 tp1/configuration_extended.py
def penalty(ps):
    return cost_9(ps) + 10 * sum(np.square(constraint_9(ps)))

In [46]:
xopt = fmin_bfgs(penalty, x0, callback=callback_9)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 36
         Function evaluations: 462
         Gradient evaluations: 42


### Solve with a constrained solver
Alternatively, the solver S-LS-QP (sequential least-square quadratic-program) optimizes over equality constraints.

In [ ]:
xopt = fmin_slsqp(cost_9, x0, callback=callback_9, f_eqcons=constraint_9, iprint=2, full_output=1)[0]

When properly defining the constraint, the solver converges quickly. It is difficult to say a-priori whether it is better to optimize with the q (and consequently a dense cost and no constraint) or with the x-y-theta (and consequently a sparse cost and constraints). Here, we empirically observe no significant difference. 